In [1]:
# %%HTML
# <style>
#     div#notebook-container    { width: 95%; }
#     div#menubar-container     { width: 65%; }
#     div#maintoolbar-container { width: 99%; }
# </style>

In [2]:
!pip install MeCab

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import sys
import itertools
import MeCab
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import collections
import itertools

In [4]:
df = pd.read_excel("特許データ.xlsx",header=0
                   ,engine='openpyxl') # engineは環境によってエラー出たり出なかったりだから、必要に応じて変える
df.head()

,公開・公表番号,発明の名称,公開・公表日,出願日,原出願日,出願人・権利者名,出願番号,登録番号,筆頭発明者,発明者名,IPC,筆頭ＩＰＣ,Ｆターム,FI,ステイタス,ST,早期請求出願有無,要約,全請求項,詳細な説明
0,登実第3082540号,Ｚ形仕切付段ボール箱,2001.12.14,2001-06-08,NaT,佐藤工業株式会社,実願2001-003729,登実第3082540号,佐藤 浩康,佐藤 浩康,B65D 5/491 (2006.01),B65D 5/491 (2006.01),"3E060AA03,3E060AB05,3E060BC02,3E060CB04,3E060C...","B65D 5/48 101J,B65D 5/498",存続期間満了による抹消,×,無,【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,【実用新案登録請求の範囲】【請求項１】 広幅と狭幅の各一対の側板を交互に横方向に連設し、それ...,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...
1,登実第3081641号,吊りフック,2001.11.16,2001-05-10,NaT,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",実願2001-002834,登実第3081641号,石塚 浩次,"石塚 浩次,織田 悦芳,好田 勝之",B66C 1/36 (2006.01),B66C 1/36 (2006.01),"3F004CC01,3F004CD08",B66C 1/36 A,存続期間満了による抹消,×,無,【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,【実用新案登録請求の範囲】【請求項１】 一端がワイヤで吊り下げ支持される少なくとも一枚のフッ...,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...
2,特開2001-311113,橋脚柱の構築工法,2001.11.09,2001-04-19,1995-06-30,株式会社大林組,特願2001-120638,NaN,大内 一,"大内 一,田中 浩一","E01D 19/02 (2006.01),E01D 21/00 (2006.01)",E01D 19/02 (2006.01),"2D059AA03,2D059CC04","E01D 19/02 ,E01D 21/04 ,E01D 21/00...",拒絶査定（１年経過）,×,無,【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提供。【構成】 構...,【特許請求の範囲】【請求項１】 フーチング部と橋梁との間に上下端を一体化させて構築される橋脚...,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...
3,特開2001-355221,コンクリートの運搬装置,2001.12.26,2001-04-06,1997-07-24,株式会社大林組,特願2001-109004,NaN,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 33/12 (2006.01)",E02B 7/00 (2006.01),"3F040BA01,3F040CA04,3F040EA03","E02B 7/00 C,B65G 33/12",拒絶査定（審判：審判請求取下）,×,有,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【特許請求の範囲】【請求項１】 回転自在に支持される両端が開口した円筒状のドラムと、前記ドラ...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬装置に...
4,特開2001-348842,コンクリートの運搬方法,2001.12.21,2001-04-06,1997-07-24,株式会社大林組,特願2001-108995,NaN,藤原 宗一,藤原 宗一,E02B 7/00 (2006.01),E02B 7/00 (2006.01),NaN,E02B 7/00 C,拒絶査定（審判：審判請求取下）,×,有,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【特許請求の範囲】【請求項１】 斜面の上端側から下端側に向けてコンクリートを運搬する方法にお...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...


In [5]:
df['詳細な説明'] = df['詳細な説明'].fillna(' ') #nanを文字列に
df['全請求項'] = df['全請求項'].fillna(' ')
df["company"] = [set(i.split(",")) for i in df['出願人・権利者名']]    #出願名をスプリットしてset型に
df['datetime'] = [dt.strptime(i.split(',')[0],  '%Y.%m.%d') for i in df['公開・公表日']]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# df['sentence'] = df['発明の名称'] +df['要約'] + df['全請求項'] .astype("str") + df['詳細な説明']  #なぜか .astype("str") いれないとエラー
df['sentence'] = df['発明の名称'] +df['要約']        #sentenceの対象にする項目も必要に応じて変える

In [6]:
#会社ごとの特許数
# pd.set_option('display.max_rows', 5000)
# pd.DataFrame(collections.Counter(itertools.chain.from_iterable(df["company"].values.tolist())).most_common())
# df.head()

,0,1
0,清水建設株式会社,6947
1,大成建設株式会社,5524
2,株式会社大林組,4940
3,鹿島建設株式会社,4668
4,株式会社竹中工務店,4565
5,株式会社熊谷組,1650
6,前田建設工業株式会社,1311
7,五洋建設株式会社,1277
8,株式会社奥村組,1222
9,株式会社フジタ,1180


In [7]:
%%time

excepword = ["上記","上部","下記","下部","前記","複数","図","要約","方法","手段","対象","現場","評価" ,"()【"]  #除外する特定の名詞
exceptype= ["接尾","副詞可能","数詞","助数詞","代名詞","複数"]  #除外する名詞のタイプ


# 
def tokenize_rm_setsubi(sentence):
    """ 連名詞のリストをトークンとして返す """
    print(".",end='') #動いてるのか不安になるからプリント
    sentence  = re.sub('\d+', '', sentence)   #数字を削除(macabでなぜか名詞に認定された数字あったからここで)
    wakati = MeCab.Tagger("")
    node = wakati.parseToNode(sentence)
    sequence = 2   #何連名詞以上をかえすか
    dictionary = []
    prev_seq = 0  #その前にいくつ名詞続いたか
    flag = 0  #
    while node:
        word = node.surface
        hinshi = node.feature.split(",")

        if prev_seq  : # その前に名詞が1以上つづいていたら
            if  (hinshi[0] == "名詞" ) and  (hinshi[0] not in exceptype) and  (hinshi[1] not in exceptype )and (hinshi[2] not in exceptype )and (word not in excepword) : #今回、除外名詞以外だったら、
                dictionary[-1] = dictionary[-1]+word   # 辞書の最後の単語にこの単語を結合
                prev_seq  +=1
                flag = 0
                
            elif (hinshi[1] == "接尾" ): #接尾辞だったら
                dictionary[-1] = dictionary[-1]+word   # とりあえず辞書の最後の単語にこの単語を結合して
                prev_seq  +=1
                flag = 1 #フラグを1に

            elif flag == 1: # 一個前が接尾辞で今度が名詞でなかったら、一個前の接尾辞で終わってる単語を辞書から削除
                dictionary.pop(-1)   #一個前の接尾辞で終わってる単語を辞書から削除
                prev_seq  =0
                flag = 0
            else:
                flag = 0
                if prev_seq  < sequence: 
                    dictionary.pop(-1) #一個前の単語を辞書から削除
                prev_seq = 0
        else:  # その前に名詞がつづいてなくて
            if  (hinshi[0] == "名詞" or hinshi[0] == "接頭辞")  and  hinshi[0] not in exceptype and  hinshi[1] not in exceptype and hinshi[2] not in exceptype and word not in excepword :
                dictionary.append(word)
                prev_seq += 1
                flag = 0

        node = node.next  
    return dictionary


# 単語ー文書行列の作成
docs = np.array(df['sentence'])

tfidf_vectorizer = TfidfVectorizer(
     tokenizer = tokenize_rm_setsubi, #上でつくった関数
    lowercase=True,
    max_df=4000,
#     smooth_idf = False,
    min_df=5)


tfidf_vectorizer.fit(docs)
terms= tfidf_vectorizer.get_feature_names()

count_vectrizer = CountVectorizer(
    vocabulary = terms,
    tokenizer = tokenize_rm_setsubi, #上でつくった関数

)
c_matrix = count_vectrizer.transform(docs).toarray()
# t_matrix = tfidf_vectorizer.transform(docs[0:10]).toarray()

/Users/WatanabeShingo/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [8]:
# len(terms)
# print(terms)

In [9]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    dic = {key: val for key, val in zip(terms, vec)  if val >=1 }
    dic2 = dict(sorted(dic.items(), key=lambda x:x[1],reverse=True))
    return dic2

df['terms'] = pd.Series([makeTermsTfDict(terms, vec) for vec in c_matrix   ])

In [10]:
# df.head()

,公開・公表番号,発明の名称,公開・公表日,出願日,原出願日,出願人・権利者名,出願番号,登録番号,筆頭発明者,発明者名,...,ステイタス,ST,早期請求出願有無,要約,全請求項,詳細な説明,company,datetime,sentence,terms
0,登実第3082540号,Ｚ形仕切付段ボール箱,2001.12.14,2001-06-08,NaT,佐藤工業株式会社,実願2001-003729,登実第3082540号,佐藤 浩康,佐藤 浩康,...,存続期間満了による抹消,×,無,【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,【実用新案登録請求の範囲】【請求項１】 広幅と狭幅の各一対の側板を交互に横方向に連設し、それ...,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...,{佐藤工業株式会社},2001-12-14,Ｚ形仕切付段ボール箱【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装...,"{'一対': 2, '横方向': 2, '連設': 2}"
1,登実第3081641号,吊りフック,2001.11.16,2001-05-10,NaT,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",実願2001-002834,登実第3081641号,石塚 浩次,"石塚 浩次,織田 悦芳,好田 勝之",...,存続期間満了による抹消,×,無,【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,【実用新案登録請求の範囲】【請求項１】 一端がワイヤで吊り下げ支持される少なくとも一枚のフッ...,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...,"{社団法人日本海上起重技術協会, 寄神建設株式会社, 東亜建設工業株式会社}",2001-11-16,吊りフック【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易か...,"{'下げ状態': 2, '回動可能': 2, '下げ支持': 1, '遠隔操作': 1}"
2,特開2001-311113,橋脚柱の構築工法,2001.11.09,2001-04-19,1995-06-30,株式会社大林組,特願2001-120638,NaN,大内 一,"大内 一,田中 浩一",...,拒絶査定（１年経過）,×,無,【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提供。【構成】 構...,【特許請求の範囲】【請求項１】 フーチング部と橋梁との間に上下端を一体化させて構築される橋脚...,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...,{株式会社大林組},2001-11-09,橋脚柱の構築工法【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提...,"{'構築工法': 3, 'コンクリート打設工程': 1, '作業足場': 1, '円形断面'..."
3,特開2001-355221,コンクリートの運搬装置,2001.12.26,2001-04-06,1997-07-24,株式会社大林組,特願2001-109004,NaN,藤原 宗一,藤原 宗一,...,拒絶査定（審判：審判請求取下）,×,有,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【特許請求の範囲】【請求項１】 回転自在に支持される両端が開口した円筒状のドラムと、前記ドラ...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬装置に...,{株式会社大林組},2001-12-26,コンクリートの運搬装置【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打...,"{'コンクリート構成材料': 3, '螺旋羽根': 3, 'コンクリートプラント': 1, ..."
4,特開2001-348842,コンクリートの運搬方法,2001.12.21,2001-04-06,1997-07-24,株式会社大林組,特願2001-108995,NaN,藤原 宗一,藤原 宗一,...,拒絶査定（審判：審判請求取下）,×,有,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【特許請求の範囲】【請求項１】 斜面の上端側から下端側に向けてコンクリートを運搬する方法にお...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...,{株式会社大林組},2001-12-21,コンクリートの運搬方法【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打...,"{'コンクリート構成材料': 3, '螺旋羽根': 3, 'コンクリートプラント': 1, ..."


In [11]:
# df.to_excel('parcedPatents211128_2.xlsx', sheet_name='Sheet1', encoding="utf-8")

In [ ]:
# ret = df.to_json('parcedPatents211128_2.json', force_ascii=False)